## Lets extract chords from a song

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce
import time
import random
import jl_io as io
import os
import uuid

In [6]:
url = 'https://tabs.ultimate-guitar.com/tab/mariah-carey/all-i-want-for-christmas-is-you-chords-903566'

In [7]:
class ChordExtractor:

    def __init__(self, raw_html_output_directory):
        self.raw_html_output_directory = raw_html_output_directory
        self.driver = self.create_chrome_driver()
        self.first_time = True
        
        if not os.path.isdir(self.raw_html_output_directory):
            os.mkdir(self.raw_html_output_directory)
            
    def extract_song_data(self,url):
        chords_spans = self.get_chord_spans(url)
        
        chords = [span.decode_contents() for span in chords_spans]
        
        song_uuid = str(uuid.uuid4())
        with open(f"{self.raw_html_output_directory}/{song_uuid}.html", "w") as file: # De los datos,como del cerdo, se guarda todo.
            file.write(self.driver.page_source )
    
        info = {
            "url":url,
            "chords":chords,
            "uuid":song_uuid
        }
        
        return info
    
    def get_chord_spans(self,url):
        self.driver.get(url)
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        article = soup.findAll('article')[3];
        
        return article.findAll('span', {"style":"color: rgb(0, 0, 0);"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver

In [8]:
extractor = ChordExtractor('../data/pages/')


In [9]:
data = extractor.extract_song_data(url)
data

{'url': 'https://tabs.ultimate-guitar.com/tab/mariah-carey/all-i-want-for-christmas-is-you-chords-903566',
 'chords': ['G',
  'Em',
  'C',
  'D',
  'G',
  'G/B',
  'C',
  'Cm/Eb',
  'G/B',
  'B7',
  'Em',
  'Cm/Eb',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'C',
  'Eb6',
  'G',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'G',
  'C',
  'Eb6',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/Eb',
  'G',
  'Em',
  'C',
  'D',
  'B7',
  'Em',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'D',
  'G',
  'C',
  'Eb6',
  'G',
  'B7',
  'Em',
  'Eb6',
  'G/D',
  'E7',
  'Am7',
  'Cm6/D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'Em',
  'C',
  'D',
  'G',
  'Em',
  'C',
  'D'],
 'uuid': 'be739ac3-2f50-4475-a977-82760d472688'}

In [ ]:
! ls ../data/pages/

## Let's prepare a place to save our data 

In [10]:
import pandas as pd
import os.path

class SongData:
    SEPARATOR = '^'
    
    def __init__(self, initial_data_path=None):
        self.df = pd.DataFrame(data=[], columns=['url','name','genre','decade','chords','uuid'])
        
        if initial_data_path is not None and os.path.isfile(initial_data_path):
            self.df = pd.read_csv(initial_data_path, sep=self.SEPARATOR)
            
    def add_basic_data(self,basic_data):
        self.df = self.df.append(basic_data,ignore_index=True)

    def add_details(self,details):
        self.df.loc[self.df['url'] == details["url"], ["chords"]] = str(details["chords"])
        self.df.loc[self.df['url'] == details["url"], ["uuid"]] = details["uuid"]
        
    def has_basic_data(self,url):
        return (self.df['url'] == url).any()
    
    def has_chords(self,url):
        return ((self.df['url'] == url) & (self.df['chords'].notnull())).any()
        
    def get_chords(self,url):
        return eval( self.df[self.df['url'] == url]['chords'][0])
    
    def has_genre_and_decade(self, genre, decade):
        return ((self.df['genre'] == genre) & (self.df['decade'] == decade)).any()
    
    def save(self, path):
        self.df.to_csv(path,index=False,sep=self.SEPARATOR)
    

        

In [ ]:
data = SongData()
data.df

In [ ]:
data.has_basic_data('http.://www.help.com')

In [ ]:
data.has_chords('http.://www.help.com')

In [ ]:
data.has_genre_and_decade('pop','1980s')

In [ ]:
data.add_basic_data({
    "name":"Help",
    "url":"http.://www.help.com",
    "genre": "pop",
    "decade": "1980s",
})

In [ ]:
data.df

In [ ]:
data.has_basic_data('http.://www.help.com')

In [ ]:
data.has_chords('http.://www.help.com')

In [ ]:
data.has_genre_and_decade('pop','1980s')

In [ ]:
data.add_details({
    "url":"http.://www.help.com",
    "chords": ["E", "F", "G"],
    'uuid': "7656-i87-gj"
})

In [ ]:
data.df

In [ ]:
data.has_basic_data('http.://www.help.com')

In [ ]:
data.has_chords('http.://www.help.com')

In [ ]:
data.get_chords('http.://www.help.com')[0]

In [ ]:
data.save('borrare.csv')

In [ ]:
data = SongData('borrare.csv')

In [ ]:
data.get_chords('http.://www.help.com')[0]

In [ ]:
! rm borrare.csv

## Link extractor (adapted from previous notebooks)

Find song links for a genre and decade

In [30]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from functools import reduce

class LinkExtractor:
    BASE_URL = "https://www.ultimate-guitar.com/explore?&type[]=Chords";
    
    def __init__(self):
        self.driver = self.create_chrome_driver()
        self.first_time = True
    
    def get_all_songs(self, genre, decade, first_page, last_page):
        links = self.get_all_filter_song_links(genre['pattern'],decade['pattern'], first_page, last_page)
        result = [ self.link_to_song_dict(link,genre,decade) for link in links]
        
        return result
    
    def link_to_song_dict(self,link, genre, decade):
        return {
            "name": link.contents[0],
            "url": link['href'],
            "genre": genre["name"],
            "decade": decade["name"]
        }
    
    def get_all_filter_song_links(self,genreFilter,decadeFilter, first_page, last_page):
        list_of_list = [self.get_links_single_page(genreFilter, decadeFilter, f'&page={page}')
                  for page in range(first_page,last_page + 1)]
        
        return reduce(lambda list1, list2: [*list1, *list2], list_of_list)
    
    def get_links_single_page(self,genreFilter,decadeFilter, pageFilter):
        self.driver.get(f'{self.BASE_URL}{genreFilter}{decadeFilter}{pageFilter}')
        
        if self.driver.page_source == '<html><head></head><body></body></html>':
            raise Exception('Denegation error')

        if self.first_time:
            self.click_on_accept_cookies()
            self.first_time = False

        soup = BeautifulSoup(self.driver.page_source, 'lxml')

        return soup.findAll('a', {"class":"_2KJtL _1mes3 kWOod"})
    
    def click_on_accept_cookies(self):
        try:
            button = self.driver.find_element_by_xpath('//button[contains(text(), "thanks")]')

            button.click()
        except:
            print('cookies banner not found. Ignored')
            
    
    def create_chrome_driver(self):
        options = webdriver.ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--incognito')
        options.add_argument('--headless')
        
        driver = webdriver.Chrome("./chromedriver", options=options)
        return driver

## Simplified extractor (adapted from previous notebooks)

We are going to iterate through genres and decades (we will forget about styles)

In [12]:
import jl_io as io
genres = io.from_file('../data/genres')
decades = io.from_file('../data/decades')

Let's extract only the first page per genre and decade

In [15]:
potential_number_of_songs = len(genres) * len(decades) * 20
potential_number_of_songs

2940

In [37]:
import jl_io as io

class SimplifiedCombinationsSongExtractor:
    def __init__(self, linkExtractor, chordExtractor, genres, decades, song_data):
        self.linkExtractor = linkExtractor
        self.chordExtractor = chordExtractor
        self.genres = genres
        self.decades = decades
        self.song_data = song_data
        
        self.combinations = []

        for decade in self.decades:
            for genre in self.genres:
                    self.combinations.append( {"decade": decade, "genre": genre})
        
    def extract(self, first_page, last_page):      
        startIndex = 1
        
        return self.extract_from(startIndex, first_page, last_page)
            
    def extract_from(self, startIndexBase1,first_page, last_page):    
        for index, combination in enumerate(self.combinations[startIndexBase1 - 1:]):
            new_extracted_songs = []
            genre = combination["genre"]
            decade = combination["decade"]
            
            if song_data.has_genre_and_decade(genre["name"], decade["name"]):
                print(f'{genre["name"]},{decade["name"]} already extracted')
                continue
            
            song_basic_data_array = []          
            try:
                song_basic_data_array = self.linkExtractor.get_all_songs(genre,decade, first_page, last_page)
                      
                for basic_data in song_basic_data_array:
                    self.song_data.add_basic_data(basic_data)
                      
                new_extracted_songs = [self.extract_song_data(index,song) for index,song in enumerate(new_extracted_songs)]
            except Exception as e: 
                print(f'Error in ({genre["name"]},{decade["name"]})')
                print(e)
                return
            
            for index,song in enumerate(song_basic_data_array):
                try:
                    self.extract_song_data(index,song)
                except Exception as e: 
                    print(f'Error in "{song["name"]}"')
                    print(e)
                    return
            
            number_of_songs = len(self.song_data.df)
            print(f'Extracted {index+startIndexBase1} of {len(self.combinations)} ({genre["name"]},{decade["name"]}). {number_of_songs} in total')  
        
    def extract_song_data(self,index,song): 
        if self.song_data.has_chords(song["url"]):
            print(f'Song {index}. "{song["name"]}" already extracted')  
            return
      
        song_details = self.chordExtractor.extract_song_data(song["url"])           
        self.song_data.add_details(song_details)
        print(f'Extracted data from song {index}. "{song["name"]}"')



## Let's use it! 

In [38]:
linkExtractor = LinkExtractor()

In [39]:
raw_html_output = '../data/raw_html'

In [40]:
chordExtractor = ChordExtractor(raw_html_output)

In [41]:
song_data = SongData()

In [42]:
extractor = SimplifiedCombinationsSongExtractor(linkExtractor,chordExtractor,genres,decades,song_data )

In [43]:
extractor.extract(first_page=1, last_page=1)

Extracted data from song 0. "All I Want"
Extracted data from song 1. "Take Me To Church (ver 2)"
Extracted data from song 2. "Radioactive"
Extracted data from song 3. "Say Something (ver 3)"
Extracted data from song 4. "Cigarette Daydreams"
Extracted data from song 5. "Believer (ver 2)"
Extracted data from song 6. "Million Reasons"
Extracted data from song 7. "Counting Stars"
Extracted data from song 8. "Sign Of The Times"
Extracted data from song 9. "Demons"
Extracted data from song 10. "Little Talks"
Extracted data from song 11. "Take Me To Church"
Extracted data from song 12. "High Hopes (ver 2)"
Extracted data from song 13. "Demons (ver 2)"
Extracted data from song 14. "Everglow"
Extracted data from song 15. "High Hopes"
Extracted data from song 16. "Say Something (ver 2)"
Extracted data from song 17. "Another Love"
Extracted data from song 18. "Pompeii (ver 3)"
Extracted data from song 19. "Breezeblocks (ver 2)"
Extracted data from song 20. "Sweet Creature"
Extracted data from son

Extracted data from song 20. "No Hard Feelings"
Extracted data from song 21. "Drink A Beer"
Extracted data from song 22. "Speak Now"
Extracted data from song 23. "Dear John (ver 4)"
Extracted data from song 24. "Mine (ver 2)"
Extracted data from song 25. "My Church"
Extracted data from song 26. "Enchanted"
Extracted data from song 27. "Flowers In Your Hair"
Extracted data from song 28. "Head Over Boots"
Extracted data from song 29. "Mean (ver 6)"
Extracted data from song 30. "Aint No Grave"
Extracted data from song 31. "Sparks Fly"
Extracted data from song 32. "Ours"
Extracted data from song 33. "Parachute"
Extracted data from song 34. "Holy"
Extracted data from song 35. "Humble And Kind (ver 2)"
Extracted data from song 36. "Slow It Down"
Extracted data from song 37. "Kansas City"
Extracted data from song 38. "Enchanted (ver 3)"
Extracted data from song 39. "Never Grow Up"
Extracted data from song 40. "Either Way"
Extracted data from song 41. "Lost In My Mind"
Extracted data from song

Extracted data from song 43. "The Morticians Daughter (ver 2)"
Extracted data from song 44. "He Is (ver 2)"
Extracted data from song 45. "Angels Fall"
Extracted data from song 46. "Dont Need You (ver 2)"
Extracted data from song 47. "Danger Line"
Extracted data from song 48. "The Call Of The Mountains"
Extracted data from song 49. "Fallen"
Extracted 50 of 147 (Metal,2010s). 350 in total
Song 0. "Love Yourself" already extracted
Song 1. "When The Partys Over" already extracted
Song 2. "I Love You" already extracted
Song 3. "Stay" already extracted
Song 4. "Bad Guy" already extracted
Song 5. "Xanny" already extracted
Song 6. "All The Good Girls Go To Hell" already extracted
Song 7. "Listen Before I Go" already extracted
Song 8. "Girls Like You" already extracted
Extracted data from song 9. "Love On The Brain"
Song 10. "Diamonds" already extracted
Song 11. "When The Partys Over (ver 2)" already extracted
Song 12. "Bury A Friend" already extracted
Song 13. "I Love You (ver 3)" already extr

Extracted data from song 13. "Green Bottles"
Extracted data from song 14. "You Better Listen"
Extracted data from song 15. "Sunshine (ver 3)"
Extracted data from song 16. "Fade Away"
Extracted data from song 17. "Let Your Hair Down"
Extracted data from song 18. "Let Your Hair Down (ver 3)"
Extracted data from song 19. "Love Comes And Goes (ver 2)"
Extracted data from song 20. "Rude (ver 8)"
Extracted data from song 21. "My Sweet Summer (ver 2)"
Extracted data from song 22. "Finest Wine"
Extracted data from song 23. "Still Young"
Extracted data from song 24. "Breeze"
Extracted data from song 25. "Molly Town"
Extracted data from song 26. "Locked Away"
Extracted data from song 27. "Augenbling"
Extracted data from song 28. "Soften The Blow"
Extracted data from song 29. "Get Free"
Extracted data from song 30. "Rude (ver 3)"
Extracted data from song 31. "Clean The House"
Extracted data from song 32. "Sound Of Change"
Extracted data from song 33. "Dont Forget Your Roots"
Extracted data from s

Extracted data from song 33. "Nada"
Extracted data from song 34. "Flor Pálida"
Extracted data from song 35. "Hipocresía"
Extracted data from song 36. "Cautivo De Este Amor"
Extracted data from song 37. "Necio"
Extracted data from song 38. "Eres Mia"
Extracted data from song 39. "Odio (ver 2)"
Extracted data from song 40. "Odio (ver 3)"
Extracted data from song 41. "Hilito"
Extracted data from song 42. "Yo Tambien"
Extracted data from song 43. "Rival"
Extracted data from song 44. "Rival (ver 2)"
Extracted data from song 45. "Challa (ver 2)"
Extracted 46 of 147 (World Music,2010s). 696 in total
Extracted data from song 0. "Get Lucky (ver 2)"
Extracted data from song 1. "Instant Crush"
Extracted data from song 2. "Get Lucky"
Extracted data from song 3. "Lose Yourself To Dance"
Extracted data from song 4. "Get Lucky (ver 4)"
Extracted data from song 5. "Touch"
Extracted data from song 6. "Instant Crush (ver 2)"
Extracted data from song 7. "Give Life Back To Music"
Extracted data from song 

...

Extracted data from song 9. "Rockin In The Free World (ver 4)"
Extracted data from song 10. "Between The Bars (ver 6)"
Extracted data from song 11. "Hand In My Pocket (ver 3)"
Extracted data from song 12. "Give Me One Reason (ver 3)"
Extracted data from song 13. "Ill Be"
Extracted data from song 14. "Two-Headed Boy"
Extracted data from song 15. "Head Over Feet"
Extracted data from song 16. "Fields Of Gold"
Extracted data from song 17. "Make You Feel My Love (ver 4)"
Extracted data from song 18. "Fields Of Gold (ver 7)"
Extracted data from song 19. "You Dont Know How It Feels"
Extracted data from song 20. "Into My Arms"
Extracted data from song 21. "You Were Meant For Me"
Extracted data from song 22. "Wildflowers (ver 2)"
Extracted data from song 23. "Angel (ver 3)"
Extracted data from song 24. "Merry Christmas From The Family"
Extracted data from song 25. "Hallelujah (ver 4)"
Extracted data from song 26. "Give Me One Reason"
Extracted data from song 27. "If It Makes You Happy"
Extracte

Extracted data from song 16. "Roads (ver 3)"
Extracted data from song 17. "Wish You Were Here"
Extracted data from song 18. "Glory Box"
Extracted data from song 19. "Sour Times"
Extracted data from song 20. "Puente"
Extracted data from song 21. "Barbie Girl (ver 4)"
Extracted data from song 22. "Its No Good"
Extracted data from song 23. "Little Black Angel"
Extracted data from song 24. "Con Un Deca"
Extracted data from song 25. "Unfinished Sympathy"
Extracted data from song 26. "Natural Blues (ver 4)"
Extracted data from song 27. "Undantag"
Extracted data from song 28. "Sister Of Night (ver 3)"
Extracted data from song 29. "Boom Boom Boom"
Extracted data from song 30. "Cosmic Girl"
Extracted data from song 31. "World In My Eyes"
Extracted data from song 32. "Come Mai (ver 2)"
Extracted data from song 33. "Half The Man"
Extracted data from song 34. "Alright"
Extracted data from song 35. "Sing It Back"
Extracted data from song 36. "Tren Al Sur"
Extracted data from song 37. "Wandering Sta

Extracted data from song 31. "Ill Make Love To You (ver 2)"
Extracted data from song 32. "Hero (ver 3)"
Extracted data from song 33. "Will You Be There"
Extracted data from song 34. "All I Want For Christmas Is You (ver 9)"
Song 35. "Ex-Factor (ver 2)" already extracted
Song 36. "Cant Take My Eyes Off You" already extracted
Extracted data from song 37. "One Last Cry (ver 4)"
Extracted data from song 38. "Jesus Born On This Day"
Extracted data from song 39. "Nothing Compares 2 U"
Song 40. "Sometimes It Snows In April (ver 2)" already extracted
Extracted data from song 41. "On Bended Knee (ver 3)"
Song 42. "I Try (ver 3)" already extracted
Song 43. "You Gotta Be (ver 3)" already extracted
Extracted data from song 44. "Give In To Me"
Song 45. "I Swear" already extracted
Extracted data from song 46. "Nice And Slow"
Extracted data from song 47. "Water Runs Dry"
Extracted data from song 48. "Miss You Most At Christmas Time"
Extracted data from song 49. "One Sweet Day"
Extracted 50 of 147 (Co

Extracted data from song 43. "Here In Your Bedroom"
Extracted data from song 44. "All I Want Is More"
Extracted data from song 45. "Burritos"
Extracted data from song 46. "Give You All The Love"
Extracted data from song 47. "The Climb"
Extracted data from song 48. "Dont Speak (ver 7)"
Extracted data from song 49. "Ñapa Es"
Extracted 50 of 147 (Reggae,1990s). 2163 in total
Extracted data from song 0. "Fields Of Gold (ver 2)"
Song 1. "I Try" already extracted
Extracted data from song 2. "Autumn Leaves"
Song 3. "Songbird (ver 2)" already extracted
Extracted data from song 4. "Chan Chan"
Extracted data from song 5. "Wade In The Water"
Song 6. "I Try (ver 3)" already extracted
Extracted data from song 7. "Dos Gardenias (ver 2)"
Extracted data from song 8. "Over The Rainbow (ver 4)"
Extracted data from song 9. "I Know You By Heart"
Extracted data from song 10. "El Cuarto De Tula"
Extracted data from song 11. "Veinte Años"
Extracted data from song 12. "Falling"
Extracted data from song 13. "F

Extracted data from song 7. "The Night Santa Went Crazy (ver 2)"
Extracted data from song 8. "Trigger Happy"
Extracted data from song 9. "Carrot Juice Is Murder"
Extracted data from song 10. "The Last Saskatchewan Pirate (ver 2)"
Extracted data from song 11. "Everything You Know Is Wrong"
Extracted data from song 12. "My Babys In Love With Eddie Vedder"
Extracted data from song 13. "Your Horoscope For Today"
Extracted data from song 14. "Your Horoscope For Today (ver 2)"
Extracted data from song 15. "Truck Drivin' Song"
Extracted data from song 16. "Grapefruit Diet"
Extracted data from song 17. "Achy Breaky Song"
Extracted data from song 18. "Traffic Jam"
Extracted data from song 19. "You Dont Love Me Anymore (ver 2)"
Extracted data from song 20. "You Dont Love Me Anymore (ver 3)"
Extracted data from song 21. "Amish Paradise"
Extracted data from song 22. "Amish Paradise (ver 2)"
Extracted data from song 23. "Callin In Sick"
Extracted data from song 24. "Since You've Been Gone"
Extracte

In [44]:
song_data.df

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['C', 'F', 'C', 'G/B', 'Am', 'F', 'C', 'F', 'C...",62d4ec07-9d59-4e00-a312-e98ce4f3b2fd
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['F#', 'Em', 'Am', 'Em', 'Am', 'G', 'Am', 'Em'...",30af0a56-524d-49b7-ac65-be4caddbe097
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G6sus2', 'Am', 'C', 'G6', 'G6sus2', 'Am', 'C...",672a1a12-3de2-4b68-bb17-55df130ce0e0
3,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['Am', 'F', 'C', 'Gsus4', 'Am', 'F', 'C', 'Gsu...",9edbfa0f-6d6d-44a4-b3d7-189d33010e15
4,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['D', 'Dmaj7', 'Em', 'G', 'A', 'D', 'Dmaj7', '...",eab426dd-9c57-4fad-b1b1-2b94601ff6ac
...,...,...,...,...,...,...
2479,https://tabs.ultimate-guitar.com/tab/warren_ze...,Lawyers Guns And Money (ver 2),Folk,1980s,NaN,NaN
2480,https://tabs.ultimate-guitar.com/tab/billy_bra...,A New England (ver 2),Folk,1980s,NaN,NaN
2481,https://tabs.ultimate-guitar.com/tab/renaud/de...,Des Que Le Vent Soufflera,Folk,1980s,NaN,NaN
2482,https://tabs.ultimate-guitar.com/tab/hannes-wa...,Es Ist An Der Zeit (ver 2),Folk,1980s,NaN,NaN


In [45]:
song_data

In [46]:
song_data.save("../data/chords.csv")

In [50]:
! ls -alt ../data/chords.csv

-rw-r--r--  1 juanluisgarcialopez  staff  1500476 Dec  8 19:14 ../data/chords.csv


In [52]:
song_data.df["decade"].unique()

array(['2010s', '2000s', '1990s', '1980s'], dtype=object)

In [58]:
len(song_data.df.groupby('url'))

2252

In [59]:
len(song_data.df.groupby(['url', 'name']))

2252

In [63]:
len(song_data.df.groupby(['url', 'name', 'decade']))

2252

In [65]:
grouped = song_data.df.groupby(['url', 'name', 'decade'])

In [69]:
grouped.count()["chords"].unique()

array([1, 0])

In [70]:
grouped.count()["genre"].unique()

array([2, 1, 3])

## Segunda extracción 

In [81]:
dataset_path = '../data/chords.csv'

In [71]:
new_song_data = SongData(dataset_path)

In [72]:
new_song_data.df

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['C', 'F', 'C', 'G/B', 'Am', 'F', 'C', 'F', 'C...",62d4ec07-9d59-4e00-a312-e98ce4f3b2fd
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['F#', 'Em', 'Am', 'Em', 'Am', 'G', 'Am', 'Em'...",30af0a56-524d-49b7-ac65-be4caddbe097
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G6sus2', 'Am', 'C', 'G6', 'G6sus2', 'Am', 'C...",672a1a12-3de2-4b68-bb17-55df130ce0e0
3,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['Am', 'F', 'C', 'Gsus4', 'Am', 'F', 'C', 'Gsu...",9edbfa0f-6d6d-44a4-b3d7-189d33010e15
4,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['D', 'Dmaj7', 'Em', 'G', 'A', 'D', 'Dmaj7', '...",eab426dd-9c57-4fad-b1b1-2b94601ff6ac
...,...,...,...,...,...,...
2479,https://tabs.ultimate-guitar.com/tab/warren_ze...,Lawyers Guns And Money (ver 2),Folk,1980s,NaN,NaN
2480,https://tabs.ultimate-guitar.com/tab/billy_bra...,A New England (ver 2),Folk,1980s,NaN,NaN
2481,https://tabs.ultimate-guitar.com/tab/renaud/de...,Des Que Le Vent Soufflera,Folk,1980s,NaN,NaN
2482,https://tabs.ultimate-guitar.com/tab/hannes-wa...,Es Ist An Der Zeit (ver 2),Folk,1980s,NaN,NaN


In [76]:
def extract(songData, first_page, last_page):
    linkExtractor = LinkExtractor()
    raw_html_output = '../data/raw_html'
    chordExtractor = ChordExtractor(raw_html_output)
    extractor = SimplifiedCombinationsSongExtractor(linkExtractor,chordExtractor,genres,decades,songData)
    extractor.extract(first_page, last_page)

In [79]:
extract(new_song_data, 1,1)

Rock,2010s already extracted
Folk,2010s already extracted
Pop,2010s already extracted
Country,2010s already extracted
Electronic,2010s already extracted
Rhythm And Blues,2010s already extracted
Metal,2010s already extracted
Contemporary R&b,2010s already extracted
Religious Music,2010s already extracted
Hip Hop,2010s already extracted
Reggae,2010s already extracted
Jazz,2010s already extracted
Blues,2010s already extracted
World Music,2010s already extracted
Disco,2010s already extracted
Comedy,2010s already extracted
New Age,2010s already extracted
Classical,2010s already extracted
Experimental,2010s already extracted
Darkwave,2010s already extracted
Soundtrack,2010s already extracted
Rock,2000s already extracted
Folk,2000s already extracted
Pop,2000s already extracted
Country,2000s already extracted
Electronic,2000s already extracted
Rhythm And Blues,2000s already extracted
Metal,2000s already extracted
Contemporary R&b,2000s already extracted
Religious Music,2000s already extracted


Extracted data from song 17. "Its Raining Men (ver 2)"
Extracted data from song 18. "Shout (ver 2)"
Extracted data from song 19. "Irgendwie Irgendwo Irgendwann"
Extracted data from song 20. "Take My Breath Away"
Extracted data from song 21. "Hungry Like The Wolf"
Extracted data from song 22. "Tainted Love (ver 5)"
Extracted data from song 23. "Sweet Dreams Are Made Of This"
Extracted data from song 24. "I Ran"
Extracted data from song 25. "Blue Monday"
Extracted data from song 26. "Take My Breath Away (ver 3)"
Extracted data from song 27. "Sunglasses At Night"
Extracted data from song 28. "Ella Elle La"
Extracted data from song 29. "Save A Prayer"
Extracted data from song 30. "The Riddle"
Extracted data from song 31. "True Faith"
Extracted data from song 32. "Never Let Me Down Again"
Extracted data from song 33. "Its Raining Men"
Extracted data from song 34. "Shake The Disease"
Extracted data from song 35. "Just Cant Get Enough (ver 2)"
Extracted data from song 36. "Rio (ver 4)"
Extrac

Extracted data from song 25. "Greatest Love Of All (ver 3)"
Extracted data from song 26. "I Just Cant Stop Loving You (ver 2)"
Extracted data from song 27. "Suddenly"
Extracted data from song 28. "Free (ver 2)"
Extracted data from song 29. "The Girl Is Mine"
Extracted data from song 30. "The Beautiful Ones"
Extracted data from song 31. "I Just Cant Stop Loving You"
Extracted data from song 32. "The Lady In My Life"
Extracted data from song 33. "Through The Fire"
Extracted data from song 34. "Starfish And Coffee"
Extracted data from song 35. "Super Freak"
Extracted data from song 36. "The Way You Make Me Feel"
Extracted data from song 37. "Billie Jean (ver 8)"
Extracted data from song 38. "Overjoyed (ver 2)"
Extracted data from song 39. "Give It To Me Baby"
Extracted data from song 40. "Through The Fire (ver 2)"
Extracted data from song 41. "I Just Cant Stop Loving You (ver 6)"
Extracted data from song 42. "The Way You Make Me Feel (ver 4)"
Extracted data from song 43. "Pyt Pretty Young

Extracted data from song 27. "Broken Bicycles (ver 2)"
Extracted data from song 28. "Little Boy Blue"
Extracted data from song 29. "You Are The Love Of My Life"
Extracted data from song 30. "Out Of Sight Out Of Mind"
Extracted data from song 31. "The Chant Has Begun"
Extracted data from song 32. "La Is My Lady"
Extracted data from song 33. "In Your Eyes"
Extracted data from song 34. "It Had To Be You (ver 4)"
Extracted data from song 35. "Breezin"
Extracted data from song 36. "Tenderly"
Extracted data from song 37. "Three Views Of A Secret"
Extracted data from song 38. "Dont Worry Be Happy (ver 6)"
Extracted 39 of 147 (Jazz,1980s). 2977 in total
Extracted data from song 0. "Josephine"
Extracted data from song 1. "On The Beach"
Extracted data from song 2. "Damn Your Eyes"
Extracted data from song 3. "I Can Hear Your Heartbeat (ver 2)"
Extracted data from song 4. "On The Beach (ver 2)"
Extracted data from song 5. "I Just Wanna Make Love To You"
Extracted data from song 6. "Stainsby Girls

Extracted data from song 3. "She Said Destroy"
Extracted data from song 4. "She Said Destroy (ver 2)"
Extracted 5 of 147 (Darkwave,1980s). 3127 in total
Extracted 84 of 147 (Soundtrack,1980s). 3127 in total
Extracted data from song 0. "Hotel California"
Extracted data from song 1. "Wish You Were Here"
Extracted data from song 2. "Imagine"
Extracted data from song 3. "Let It Be (ver 2)"
Extracted data from song 4. "Your Song"
Extracted data from song 5. "Knockin On Heavens Door (ver 2)"
Extracted data from song 6. "Bohemian Rhapsody (ver 5)"
Extracted data from song 7. "Have You Ever Seen The Rain"
Extracted data from song 8. "Love Of My Life"
Extracted data from song 9. "Let It Be"
Extracted data from song 10. "Father And Son"
Extracted data from song 11. "Sweet Caroline"
Extracted data from song 12. "Dont Stop Me Now (ver 3)"
Extracted data from song 13. "Heart Of Gold"
Extracted data from song 14. "Piano Man (ver 5)"
Extracted data from song 15. "Please Come Home For Christmas"
Extra

Extracted data from song 6. "Always On My Mind"
Song 7. "Knockin On Heavens Door (ver 5)" already extracted
Extracted data from song 8. "Sunday Morning Coming Down (ver 5)"
Song 9. "Pancho And Lefty (ver 2)" already extracted
Extracted data from song 10. "Ghost Riders In The Sky (ver 7)"
Song 11. "Take Me Home Country Roads (ver 3)" already extracted
Extracted data from song 12. "Listen To The Music (ver 4)"
Extracted data from song 13. "Rocky Mountain High"
Extracted data from song 14. "Help Me Make It Through The Night (ver 2)"
Extracted data from song 15. "Friend Of The Devil (ver 2)"
Extracted data from song 16. "I Will Always Love You (ver 4)"
Extracted data from song 17. "Blue Eyes Crying In The Rain"
Extracted data from song 18. "Angel From Montgomery (ver 2)"
Extracted data from song 19. "Margaritaville (ver 2)"
Extracted data from song 20. "Knockin On Heavens Door"
Extracted data from song 21. "Long Train Runnin (ver 2)"
Extracted data from song 22. "Beautiful Star Of Bethlehe

In [80]:
new_song_data.df

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['C', 'F', 'C', 'G/B', 'Am', 'F', 'C', 'F', 'C...",62d4ec07-9d59-4e00-a312-e98ce4f3b2fd
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['F#', 'Em', 'Am', 'Em', 'Am', 'G', 'Am', 'Em'...",30af0a56-524d-49b7-ac65-be4caddbe097
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G6sus2', 'Am', 'C', 'G6', 'G6sus2', 'Am', 'C...",672a1a12-3de2-4b68-bb17-55df130ce0e0
3,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['Am', 'F', 'C', 'Gsus4', 'Am', 'F', 'C', 'Gsu...",9edbfa0f-6d6d-44a4-b3d7-189d33010e15
4,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['D', 'Dmaj7', 'Em', 'G', 'A', 'D', 'Dmaj7', '...",eab426dd-9c57-4fad-b1b1-2b94601ff6ac
...,...,...,...,...,...,...
3396,https://tabs.ultimate-guitar.com/tab/van_morri...,Moondance (ver 4),Rhythm And Blues,1970s,NaN,NaN
3397,https://tabs.ultimate-guitar.com/tab/hall-oate...,Rich Girl,Rhythm And Blues,1970s,NaN,NaN
3398,https://tabs.ultimate-guitar.com/tab/hall-oate...,Sara Smile,Rhythm And Blues,1970s,NaN,NaN
3399,https://tabs.ultimate-guitar.com/tab/marvin-ga...,Lets Get It On,Rhythm And Blues,1970s,NaN,NaN


In [82]:
new_song_data.save(dataset_path)

## Tercera extracción 

In [83]:
song_data_3 = SongData(dataset_path)

In [84]:
song_data_3.df

,url,name,genre,decade,chords,uuid
0,https://tabs.ultimate-guitar.com/tab/kodaline/...,All I Want,Rock,2010s,"['C', 'F', 'C', 'G/B', 'Am', 'F', 'C', 'F', 'C...",62d4ec07-9d59-4e00-a312-e98ce4f3b2fd
1,https://tabs.ultimate-guitar.com/tab/hozier/ta...,Take Me To Church (ver 2),Rock,2010s,"['F#', 'Em', 'Am', 'Em', 'Am', 'G', 'Am', 'Em'...",30af0a56-524d-49b7-ac65-be4caddbe097
2,https://tabs.ultimate-guitar.com/tab/imagine-d...,Radioactive,Rock,2010s,"['G6sus2', 'Am', 'C', 'G6', 'G6sus2', 'Am', 'C...",672a1a12-3de2-4b68-bb17-55df130ce0e0
3,https://tabs.ultimate-guitar.com/tab/a_great_b...,Say Something (ver 3),Rock,2010s,"['Am', 'F', 'C', 'Gsus4', 'Am', 'F', 'C', 'Gsu...",9edbfa0f-6d6d-44a4-b3d7-189d33010e15
4,https://tabs.ultimate-guitar.com/tab/cage-the-...,Cigarette Daydreams,Rock,2010s,"['D', 'Dmaj7', 'Em', 'G', 'A', 'D', 'Dmaj7', '...",eab426dd-9c57-4fad-b1b1-2b94601ff6ac
...,...,...,...,...,...,...
3396,https://tabs.ultimate-guitar.com/tab/van_morri...,Moondance (ver 4),Rhythm And Blues,1970s,NaN,NaN
3397,https://tabs.ultimate-guitar.com/tab/hall-oate...,Rich Girl,Rhythm And Blues,1970s,NaN,NaN
3398,https://tabs.ultimate-guitar.com/tab/hall-oate...,Sara Smile,Rhythm And Blues,1970s,NaN,NaN
3399,https://tabs.ultimate-guitar.com/tab/marvin-ga...,Lets Get It On,Rhythm And Blues,1970s,NaN,NaN


In [85]:
extract(song_data_3,1,1)

Rock,2010s already extracted
Folk,2010s already extracted
Pop,2010s already extracted
Country,2010s already extracted
Electronic,2010s already extracted
Rhythm And Blues,2010s already extracted
Metal,2010s already extracted
Contemporary R&b,2010s already extracted
Religious Music,2010s already extracted
Hip Hop,2010s already extracted
Reggae,2010s already extracted
Jazz,2010s already extracted
Blues,2010s already extracted
World Music,2010s already extracted
Disco,2010s already extracted
Comedy,2010s already extracted
New Age,2010s already extracted
Classical,2010s already extracted
Experimental,2010s already extracted
Darkwave,2010s already extracted
Soundtrack,2010s already extracted
Rock,2000s already extracted
Folk,2000s already extracted
Pop,2000s already extracted
Country,2000s already extracted
Electronic,2000s already extracted
Rhythm And Blues,2000s already extracted
Metal,2000s already extracted
Contemporary R&b,2000s already extracted
Religious Music,2000s already extracted


Song 0. "I Just Called To Say I Love You (ver 4)" already extracted
Song 1. "Hello (ver 5)" already extracted
Song 2. "Everybody Needs Somebody To Love" already extracted
Song 3. "Proud Mary" already extracted
Song 4. "Wake Me Up Before You Go-Go" already extracted
Song 5. "Stuck On You (ver 4)" already extracted
Song 6. "Unchain My Heart" already extracted
Song 7. "Nothings Gonna Change My Love For You (ver 4)" already extracted
Song 8. "All Night Long" already extracted
Song 9. "Whats Love Got To Do With It (ver 2)" already extracted
Song 10. "Sweet Home Chicago (ver 2)" already extracted
Song 11. "Holding Back The Years" already extracted
Song 12. "Hello" already extracted
Song 13. "Smooth Operator" already extracted
Song 14. "Maneater" already extracted
Song 15. "Out Of Touch (ver 2)" already extracted
Song 16. "Im So Excited (ver 2)" already extracted
Song 17. "Every Time You Go Away (ver 2)" already extracted
Song 18. "You Make My Dreams (ver 4)" already extracted
Song 19. "That 

Song 0. "Just A Friend" already extracted
Song 1. "Rappers Delight" already extracted
Song 2. "Ice Ice Baby" already extracted
Song 3. "Its Tricky" already extracted
Extracted 4 of 147 (Hip Hop,1980s). 3755 in total
Song 0. "Redemption Song (ver 2)" already extracted
Song 1. "Redemption Song" already extracted
Song 2. "Redemption Song (ver 3)" already extracted
Song 3. "Red Red Wine" already extracted
Song 4. "Could You Be Loved" already extracted
Song 5. "Natural Mystic" already extracted
Song 6. "Kingston Town" already extracted
Song 7. "It Must Be Love (ver 5)" already extracted
Song 8. "Kingston Town (ver 2)" already extracted
Song 9. "Red Red Wine (ver 3)" already extracted
Song 10. "Our House (ver 3)" already extracted
Song 11. "Lively Up Yourself" already extracted
Song 12. "Your House" already extracted
Song 13. "Soul Rebel" already extracted
Song 14. "Sinds 1 Dag Of 2 32 Jaar" already extracted
Song 15. "Ghost Town" already extracted
Song 16. "I Got You Babe" already extracted

Song 0. "Yoda" already extracted
Song 1. "Melanie" already extracted
Song 2. "I Love Rocky Road" already extracted
Song 3. "Christmas At Ground Zero" already extracted
Song 4. "One More Minute (ver 3)" already extracted
Song 5. "Good Enough For Now" already extracted
Song 6. "Stop Draggin My Car Around" already extracted
Song 7. "I Think Im A Clone Now" already extracted
Song 8. "My Bologna" already extracted
Song 9. "Another One Rides The Bus" already extracted
Song 10. "Ricky" already extracted
Song 11. "Midnight Star" already extracted
Song 12. "This Is The Life" already extracted
Song 13. "Living With A Hernia" already extracted
Song 14. "You Make Me" already extracted
Song 15. "The Biggest Ball Of Twine In Minnesota" already extracted
Song 16. "Generic Blues" already extracted
Song 17. "Eat It" already extracted
Song 18. "One More Minute" already extracted
Song 19. "One More Minute (ver 2)" already extracted
Song 20. "Dare To Be Stupid" already extracted
Song 21. "I Lost On Jeopar

Song 0. "Take Me Home Country Roads (ver 10)" already extracted
Song 1. "Take Me Home Country Roads" already extracted
Song 2. "Knockin On Heavens Door (ver 2)" already extracted
Song 3. "Jolene (ver 2)" already extracted
Song 4. "The Gambler (ver 2)" already extracted
Song 5. "Take Me Home Country Roads (ver 2)" already extracted
Song 6. "Always On My Mind" already extracted
Song 7. "Knockin On Heavens Door (ver 5)" already extracted
Song 8. "Sunday Morning Coming Down (ver 5)" already extracted
Song 9. "Pancho And Lefty (ver 2)" already extracted
Song 10. "Ghost Riders In The Sky (ver 7)" already extracted
Song 11. "Take Me Home Country Roads (ver 3)" already extracted
Song 12. "Listen To The Music (ver 4)" already extracted
Song 13. "Rocky Mountain High" already extracted
Song 14. "Help Me Make It Through The Night (ver 2)" already extracted
Song 15. "Friend Of The Devil (ver 2)" already extracted
Song 16. "I Will Always Love You (ver 4)" already extracted
Song 17. "Blue Eyes Crying

...

Extracted data from song 32. "Ode To Billie Joe (ver 5)"
Extracted data from song 33. "It Aint Me Babe (ver 2)"
Extracted data from song 34. "In The Ghetto (ver 5)"
Extracted data from song 35. "Cocaine Blues (ver 2)"
Extracted data from song 36. "Walkin After Midnight"
Extracted data from song 37. "Keep On The Sunny Side (ver 3)"
Extracted data from song 38. "Tennessee Flat Top Box"
Extracted data from song 39. "Fare Thee Well Miss Carousel"
Extracted data from song 40. "Tecumseh Valley (ver 4)"
Extracted data from song 41. "Danny Boy"
Extracted data from song 42. "I Can Hear Music"
Extracted data from song 43. "Gentle On My Mind"
Extracted data from song 44. "Gentle On My Mind (ver 2)"
Extracted data from song 45. "Galveston"
Extracted data from song 46. "Crazy (ver 4)"
Extracted data from song 47. "Lungs"
Extracted data from song 48. "San Quentin (ver 2)"
Extracted data from song 49. "Wichita Lineman (ver 3)"
Extracted 50 of 147 (Country,1960s). 4789 in total
Extracted data from son

Extracted data from song 10. "Fly Me To The Moon (ver 4)"
Song 11. "Both Sides Now" already extracted
Song 12. "What A Wonderful World" already extracted
Song 13. "Both Sides Now (ver 3)" already extracted
Extracted data from song 14. "Balladen Om Herr Fredrik Åkare Och Den Söta Fröken Cecilia Lind (ver 4)"
Extracted data from song 15. "Love (ver 2)"
Song 16. "Aint Got No I Got Life" already extracted
Extracted data from song 17. "Guantanamera"
Extracted data from song 18. "Fly Me To The Moon (ver 8)"
Extracted data from song 19. "What Are You Doing New Years Eve"
Song 20. "You Dont Know Me (ver 3)" already extracted
Extracted data from song 21. "Strangers In The Night"
Extracted data from song 22. "Georgia On My Mind"
Extracted data from song 23. "Misty"
Extracted data from song 24. "Les Copains Dabord (ver 2)"
Extracted data from song 25. "I Put A Spell On You"
Extracted data from song 26. "Sinnerman"
Extracted data from song 27. "Fly Me To The Moon (ver 5)"
Extracted data from song 

Extracted data from song 28. "Bye Bye Love (ver 2)"
Extracted data from song 29. "Dont Be Cruel (ver 4)"
Extracted data from song 30. "Jailhouse Rock (ver 5)"
Extracted data from song 31. "Wake Up Little Susie"
Extracted data from song 32. "Cry Cry Cry"
Extracted data from song 33. "Great Balls Of Fire (ver 2)"
Extracted data from song 34. "Blue Suede Shoes (ver 2)"
Extracted data from song 35. "Johnny B Goode (ver 4)"
Extracted data from song 36. "Blue Christmas (ver 4)"
Extracted data from song 37. "Give My Love To Rose"
Extracted data from song 38. "Hello Mary Lou (ver 5)"
Extracted data from song 39. "White Christmas"
Extracted data from song 40. "Summertime Blues (ver 2)"
Extracted data from song 41. "Rave On"
Extracted data from song 42. "Bye Bye Love"
Extracted data from song 43. "Dont Be Cruel (ver 5)"
Extracted data from song 44. "Hound Dog (ver 3)"
Extracted data from song 45. "Big River (ver 2)"
Extracted data from song 46. "Dont Take Your Guns To Town"
Extracted data from s

Extracted data from song 35. "Cool Water"
Extracted data from song 36. "Theyre Hanging Me Tonight (ver 2)"
Extracted data from song 37. "Folsom Prison Blues (ver 3)"
Extracted data from song 38. "My Buckets Got A Hole In It"
Extracted data from song 39. "When Will I Be Loved"
Extracted data from song 40. "Oh Lonesome Me"
Extracted data from song 41. "Its Only Make Believe (ver 2)"
Extracted data from song 42. "I Got Stripes"
Extracted data from song 43. "The Battle Of New Orleans (ver 2)"
Extracted data from song 44. "Lovesick Blues (ver 3)"
Extracted data from song 45. "Walking After Midnight (ver 3)"
Extracted data from song 46. "Its Only Make Believe (ver 4)"
Extracted data from song 47. "El Paso"
Extracted data from song 48. "I Still Miss Someone (ver 2)"
Extracted data from song 49. "Home Of The Blues"
Extracted 50 of 147 (Country,1950s). 5241 in total
Extracted 131 of 147 (Electronic,1950s). 5241 in total
Song 0. "Love Me Tender" already extracted
Song 1. "Heartbreak Hotel" alrea

Extracted 141 of 147 (Disco,1950s). 5375 in total
Extracted 142 of 147 (Comedy,1950s). 5375 in total
Extracted 143 of 147 (New Age,1950s). 5375 in total
Extracted data from song 0. "The Water Is Wide"
Extracted 1 of 147 (Classical,1950s). 5376 in total
Extracted 145 of 147 (Experimental,1950s). 5376 in total
Extracted 146 of 147 (Darkwave,1950s). 5376 in total
Extracted 147 of 147 (Soundtrack,1950s). 5376 in total


In [137]:
len(song_data_3.df)

5376

In [138]:
song_data_3.save(dataset_path)